In [1]:
import json
import os
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import keras
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

from keras.layers import merge
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *

import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
import gensim.models.keyedvectors as word2vec
from gensim.models.keyedvectors import KeyedVectors

glove2word2vec(glove_input_file="glove.6B.50d.txt", word2vec_output_file="gensim_glove_vectors.txt")
glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def filter_sentence(text):
    #tokenizer = RegexpTokenizer(r'\w+')
    tokenizer = RegexpTokenizer(r'[A-z]+')
    stop_words = set(stopwords.words('english'))
    word_tokens = [string.lower() for string in tokenizer.tokenize(text)]
    word_tokens = [w for w in word_tokens if not w in stop_words]
    #word_tokens = [w for w in word_tokens if len(w)>2]
    return word_tokens

In [3]:
folder_names = ["charliehebdo","ferguson","germanwings-crash","ottawashooting","sydneysiege"]

In [4]:
non_rumour_json_obj = []
for name in folder_names:
    non_rumours_paths = "pheme-rnr-dataset/"+name+"/non-rumours/"
    non_rumours_path = [x for x in os.listdir(non_rumours_paths)]
    for subfolder in non_rumours_path:
        subfolder_path = non_rumours_paths + subfolder + "/source-tweet/"
        json_files = [pos_json for pos_json in os.listdir(subfolder_path) if pos_json.endswith('.json')]
        for json_file in json_files:
            json_file_path = subfolder_path + json_file
            #print(json_file_path)
            with open(json_file_path) as json_data:
                d = json.load(json_data)
                non_rumour_json_obj.append(d)
rumour_json_obj = []
for name in folder_names:
    rumours_paths = "pheme-rnr-dataset/"+name+"/rumours/"
    rumours_path = [x for x in os.listdir(rumours_paths)]
    for subfolder in rumours_path:
        subfolder_path = rumours_paths + subfolder + "/source-tweet/"
        json_files = [pos_json for pos_json in os.listdir(subfolder_path) if pos_json.endswith('.json')]
        for json_file in json_files:
            json_file_path = subfolder_path + json_file
            #print(json_file_path)
            with open(json_file_path) as json_data:
                d = json.load(json_data)
                rumour_json_obj.append(d)

In [5]:
retweet_count = []
text = []
created = []
followers_count = []
verified = []
statuses_count = []
user_age = []
rumour = []

for elem in non_rumour_json_obj:
    retweet_count.append(elem['retweet_count'])
    text.append(elem['text'])
    created.append(elem['created_at'])
    followers_count.append(elem['user']['followers_count'])
    verified.append(elem['user']['verified'])
    statuses_count.append(elem['user']['statuses_count'])
    user_age.append(elem['user']['created_at'])
    rumour.append('non_rumour')
for elem in rumour_json_obj:
    retweet_count.append(elem['retweet_count'])
    text.append(elem['text'])
    created.append(elem['created_at'])
    followers_count.append(elem['user']['followers_count'])
    verified.append(elem['user']['verified'])
    statuses_count.append(elem['user']['statuses_count'])
    user_age.append(elem['user']['created_at'])
    rumour.append('rumour')

In [6]:
df_temp = pd.DataFrame()
df = pd.DataFrame()

df_temp['retweet_count'] = retweet_count
df_temp['text'] = text
df_temp['created'] = created
df_temp['followers_count'] = followers_count
df_temp['verified'] = verified
df_temp['statuses_count'] = statuses_count
df_temp['user_age'] = user_age
df_temp['rumour'] = rumour

In [7]:
df.count()
df_temp.to_csv('/home/dell/rumour.csv',sep=',')

In [8]:
le = preprocessing.LabelEncoder()
df_temp['verified_bool'] = le.fit_transform(df_temp['verified'])
df_temp['rumour_bool'] = le.fit_transform(df_temp['rumour'])
actual_labels = df_temp['rumour_bool']

In [9]:
df = df_temp[['retweet_count','followers_count','verified_bool','statuses_count']]

In [10]:
clf = RandomForestClassifier(max_depth=2)
X_train, X_test, y_train, y_test = train_test_split(df, actual_labels, test_size=0.33, random_state=42)

In [11]:
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(accuracy_score(pred,y_test))

0.658485639686684


In [12]:
####next attempt

In [13]:
tokens = []
for t in text:
    token_line = filter_sentence(t)
    tokens.append(token_line)
word_features = []

not_found = 0
text_features = []
for token_line in tokens:
    line = []
    for token in token_line:
            try: 
                word_vector = glove_model.get_vector(token)
                line.append(word_vector)
            except:
                not_found = not_found + 1
    text_features.append(line)

In [14]:
##take average of features
avg_text_features = []
for lines in text_features:
    avg = np.zeros(50)
    count = 0
    for line in lines:
        avg = avg + line
        count = count + 1
    avg_text_features.append(avg)

In [15]:
word_avg_features = np.array(avg_text_features)
concat_features = np.concatenate((word_avg_features,df.as_matrix()),axis=1)

In [16]:
clf = RandomForestClassifier(max_depth=2)
X_train, X_test, y_train, y_test = train_test_split(word_avg_features, actual_labels, test_size=0.33, random_state=42)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(accuracy_score(pred,y_test))

0.7044386422976502


In [17]:
clf = RandomForestClassifier(max_depth=2)
X_train, X_test, y_train, y_test = train_test_split(concat_features, actual_labels, test_size=0.33, random_state=42)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(accuracy_score(pred,y_test))

0.7086161879895562


In [18]:
####LSTM approach

In [19]:
#x = Input(shape=(32,))
#y = Dense(16, activation='softmax')(x)
#model = Model(x, y)
lengths = []
for line in text_features:
    lengths.append(len(line))
print(max(lengths))

21


In [20]:
len(text_features)

5802

In [21]:
lengths = []
training_list_embedded = np.zeros(shape=(len(text_features),50,15))
for i in range(len(text_features)):
    sentence = text_features[i]
    length = len(sentence)
    padded_sequence = []
    j = 1
        
    while(1):
            #print(j)
            if j>(15-length):
                break
            padded_sequence.append(np.zeros(50))
            j = j + 1
    sentence = padded_sequence + sentence 
    #print(length,15-length)
    #print(np.array(sentence).transpose().shape)
    training_list_embedded[i,:,:] = np.array(sentence).transpose()[:,:15]
    #print(str(i) + " " + str(len(sentence)))

In [22]:
X_train, X_test, y_train, y_test = train_test_split(training_list_embedded, actual_labels, test_size=0.33, random_state=42)

In [23]:
# create the model
model = Sequential()
model.add(LSTM(10,input_shape=(50,15,)))
model.add(Dense(10))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(training_list_embedded, np.array(actual_labels), nb_epoch=30, batch_size=100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10)                1040      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 1,161
Trainable params: 1,161
Non-trainable params: 0
_________________________________________________________________
None


/home/dell/.local/lib/python3.5/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/30
5802/5802 [==============================] - 10s 2ms/step - loss: 0.6693 - acc: 0.6124
Epoch 2/30
5802/5802 [==============================] - 3s 445us/step - loss: 0.6400 - acc: 0.6601
Epoch 3/30
5802/5802 [==============================] - 2s 421us/step - loss: 0.6344 - acc: 0.6603
Epoch 4/30
5802/5802 [==============================] - 2s 421us/step - loss: 0.6300 - acc: 0.6610
Epoch 5/30
5802/5802 [==============================] - 2s 417us/step - loss: 0.6241 - acc: 0.6639
Epoch 6/30
5802/5802 [==============================] - 2s 416us/step - loss: 0.6083 - acc: 0.6713
Epoch 7/30
5802/5802 [==============================] - 2s 417us/step - loss: 0.5779 - acc: 0.6942
Epoch 8/30
5802/5802 [==============================] - 2s 415us/step - loss: 0.5613 - acc: 0.7075
Epoch 9/30
5802/5802 [==============================] - 2s 416us/step - loss: 0.5512 - acc: 0.7051
Epoch 10/30
5802/5802 [==============================] - 2s 417us/step - loss: 0.5448 - acc: 0.7073
Epoch 11/3

In [24]:
model.evaluate(X_test,y_test)

1915/1915 [==============================] - 1s 342us/step


[0.49819658263233874, 0.7524804174121926]

In [25]:
df.head()

,retweet_count,followers_count,verified_bool,statuses_count
0,710,12933705,1,22815
1,417,36137,0,4628
2,103,453,0,13825
3,319,973003,1,124056
4,157,193927,1,63586


In [26]:
df_temp

,retweet_count,text,created,followers_count,verified,statuses_count,user_age,rumour,verified_bool,rumour_bool
0,710,Video shows police surrounding Paris kosher su...,Fri Jan 09 13:16:57 +0000 2015,12933705,True,22815,Sun Apr 22 14:42:37 +0000 2007,non_rumour,1,0
1,417,French Justice Minister C.Taubira reacts outsi...,Wed Jan 07 13:47:53 +0000 2015,36137,False,4628,Mon Oct 08 14:43:38 +0000 2012,non_rumour,0,0
2,103,"If you kill men for making fun of your ideas, ...",Wed Jan 07 12:47:12 +0000 2015,453,False,13825,Wed Jun 24 05:08:56 +0000 2009,non_rumour,0,0
3,319,French media calling #CharlieHebdo attack the ...,Wed Jan 07 13:22:59 +0000 2015,973003,True,124056,Sun Oct 26 00:00:29 +0000 2008,non_rumour,1,0
4,157,Israeli SWAT team unit specializing in hostage...,Fri Jan 09 15:15:10 +0000 2015,193927,True,63586,Sun Mar 15 09:43:07 +0000 2009,non_rumour,1,0
5,142,THE UTTER HYPOCRISY\n\n#JeSuisCharlie http://t...,Fri Jan 09 11:31:14 +0000 2015,150967,True,106599,Sat Mar 19 16:40:35 +0000 2011,non_rumour,1,0
6,102,"#CharlieHebdo’s Charb: ""I would rather die sta...",Wed Jan 07 14:18:25 +0000 2015,22572,False,55260,Sat Sep 26 17:37:32 +0000 2009,non_rumour,0,0
7,791,"#JeSuisCharlie Hey folks, enter for free payp...",Thu Jan 08 16:47:48 +0000 2015,20,False,395,Mon Sep 22 19:12:57 +0000 2014,non_rumour,0,0
8,118,Hero police officer executed in street was 42-...,Thu Jan 08 09:50:07 +0000 2015,818286,True,125631,Tue Jul 15 10:17:46 +0000 2008,non_rumour,1,0
9,130,"""But who wants the English in Europe?"" Quite e...",Wed Jan 07 14:42:00 +0000 2015,7105,False,11087,Thu Sep 17 17:06:55 +0000 2009,non_rumour,0,0
